# < 알파벳을 기준으로 그룹별로 정렬하기 >
    * 전달받은 데이터는 외국인 셀럽들의 데이터이다. 회사에서 요구한 것은 각 셀럽들의 성이 아닌 이름의 맨 앞 알파벳을 기준으로 그룹을 지어달라는 것! (+ 이름대신 공유 인덱스 넘버로 )
예 : Amanda Seyfried 는 Actor 그룹의  'S'그룹

1) 성이 아닌 이름을 기준으로 할 것
    - 예외 케이스 고려하기 ( 미들네임 or '.'이 이름에 있는 경우 등)

2) 정렬 기준은 그룹 -> 이름의 알파벳. 각각 인물들의 직업이 우선순위로 그룹화 되어있어야 함



In [1]:
import pandas as pd
import numpy as np
import re

# 전달받은 data set
data = pd.read_excel("/Users/myhellebs/Downloads/키워드 카테고리 구분 작업.xlsx")#.drop('kl_name_en',axis=1)
data

,ks_pk,keyword_name,kl_name_en,type
0,1,Mark Hamill,Mark Hamill,Actor
1,2,Harrison Ford,Harrison Ford,Actor
2,3,Peter Cushing,Peter Cushing,Actor
3,4,Anthony Daniels,Anthony Daniels,Actor
4,5,Bob Peterson,Bob Peterson,Actor
...,...,...,...,...
6845,6846,Susanna Fogel,Susanna Fogel,Director : Female
6846,6847,Julia Ducournau,Julia Ducournau,Director : Female
6847,6848,Lucia Aniello,Lucia Aniello,Director : Female
6848,6849,Stella Meghie,Stella Meghie,Director : Female


## < 기준 및 예외 케이스 정하기 >
    - 성 + 이름(두개의 단어)으로 되어 있는 사람은 이름의 맨 앞글자 대문자만 추출
    - 미들네임이 있는 경우에는 맨 뒤에 있는 이름으로 선택. 예)Jennifer Yuh Nelson = N그룹
    - 이름에 Junior or Jr가 있는 경우에는, jr바로 이전 단어를 이름으로 선택. 예)John Gallagher Jr. = G그룹
    - 이름에 특수문자('.','-' etc)가 있는건 특이 케이스 이므로 주의해서 따로 확인할 것

In [2]:
global check_list
check_list = []

def Upper_Group(x):
    
    unit = x.split(" ")
    Alphabet = re.compile('[A-Z]') # 대문자만 추출  
    sc = re.compile("[^a-zA-Z\s]") # 공백을 제외한 특수문자가 있는 이름들의 확인을 위한 정규식
    
    upper = Alphabet.findall(x)
    
    try :  
        if ('Jr.' in unit) or ('Junior' in unit):
            return upper[-2]
        
        else : # stereo type의 모든 이름(성 + 이름)
            return upper[-1] 
        
    finally :     
        if sc.findall(x): # 특수문자가 있으면
            check_list.append(x)    


In [3]:
# 이름의 대문자를 추출한 새로운 칼럼 생성
data['upper'] = data['keyword_name'].apply(Upper_Group)
data

,ks_pk,keyword_name,kl_name_en,type,upper
0,1,Mark Hamill,Mark Hamill,Actor,H
1,2,Harrison Ford,Harrison Ford,Actor,F
2,3,Peter Cushing,Peter Cushing,Actor,C
3,4,Anthony Daniels,Anthony Daniels,Actor,D
4,5,Bob Peterson,Bob Peterson,Actor,P
...,...,...,...,...,...
6845,6846,Susanna Fogel,Susanna Fogel,Director : Female,F
6846,6847,Julia Ducournau,Julia Ducournau,Director : Female,D
6847,6848,Lucia Aniello,Lucia Aniello,Director : Female,A
6848,6849,Stella Meghie,Stella Meghie,Director : Female,M


In [4]:
# 특수문자가 있던 이름들을 조회 후
data[data['keyword_name'].isin(check_list)].iloc[0:50]
# ks_pk = 191 , 227 ,243, 383 ,... - 특이 데이터 확인 그리고 수정하기
check = [191 , 227 ,243, 383 ]
data[data['ks_pk'].isin(check)]

,ks_pk,keyword_name,kl_name_en,type,upper
190,191,Rade Šerbedžija,Rade Šerbedžija,Actor,R
226,227,Tony Leung Chiu-Wai,Tony Leung Chiu-Wai,Actor,W
242,243,Brian Doyle-Murray,Brian Doyle-Murray,Actor,M
382,383,David O'Hara,David O'Hara,Actor,H


In [5]:
# 특이 데이터의 알파벳 그룹이 수정됐음을 확인
data.loc[data.ks_pk == 191, 'upper'] = "S"
data.loc[data.ks_pk == 227, 'upper'] = "C"
data.loc[data.ks_pk == 243, 'upper'] = "D"
data.loc[data.ks_pk == 383, 'upper'] = "O"

data[data['ks_pk'].isin(check)]

,ks_pk,keyword_name,kl_name_en,type,upper
190,191,Rade Šerbedžija,Rade Šerbedžija,Actor,S
226,227,Tony Leung Chiu-Wai,Tony Leung Chiu-Wai,Actor,C
242,243,Brian Doyle-Murray,Brian Doyle-Murray,Actor,D
382,383,David O'Hara,David O'Hara,Actor,O


In [6]:
alphabet_group = list(data['upper'].unique())
alphabet_group = sorted(alphabet_group)
type_group = list(data['type'].unique())

In [10]:
final = []

for job in type_group :
    
    for alpha in alphabet_group :
        
        pk_list = list(data[(data['type'] == job) & (data['upper'] == alpha)]['ks_pk'])
        pk_list = str(pk_list).replace("[","").replace("]","")
        
        names = list(data[(data['type'] == job) & (data['upper'] == alpha)]['keyword_name'])
        names = str(names).replace("[","").replace("]","").replace("\'","")
        
        result = {'type':job ,'upper':alpha, 'pk_num':pk_list, 'celeb':names}
        
        final.append(result)
        

In [11]:
# 최종 결과
pd.DataFrame(final)

,type,upper,pk_num,celeb
0,Actor,A,"19, 122, 134, 150, 163, 170, 171, 198, 202, 22...","Alfred Abel, Dan Aykroyd, John Ashton, Ben Aff..."
1,Actor,B,"7, 12, 30, 32, 38, 58, 76, 82, 85, 90, 98, 102...","Albert Brooks, Sean Bean, Orlando Bloom, Kenny..."
2,Actor,C,"3, 36, 44, 45, 47, 83, 86, 87, 126, 135, 158, ...","Peter Cushing, David Carradine, Dan Castellane..."
3,Actor,D,"4, 24, 53, 113, 184, 232, 238, 243, 244, 257, ...","Anthony Daniels, Johnny Depp, Keir Dullea, Joh..."
4,Actor,E,"40, 59, 63, 114, 204, 242, 291, 304, 322, 416,...","Clint Eastwood, Emilio Echevarría, Eminem, Den..."
...,...,...,...,...
99,Director : Female,V,6824,Agnieszka Wojtowicz-Vosloo
100,Director : Female,W,"6778, 6779, 6806, 6823, 6829, 6832, 6837","Lilly Wachowski, Lana Wachowski, Olivia Wilde,..."
101,Director : Female,X,,
102,Director : Female,Y,6836,Naoko Yamada
